
<h1>Import packages</h1>

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
 
import pandas as pd
 
import matplotlib.pyplot as plt
%matplotlib inline
 
import numpy as np
 
from tqdm import tqdm 
import time

from IPython.display import clear_output
 
use_cuda = True
if torch.cuda.is_available():
    print("CUDA is available!")
    device = torch.device("cuda")

<h1>Data organization</h1>

In [ ]:
class GenDataset(Dataset):
 
  def __init__(self, X, transform=None):
      self.X = X.float()
      self.data = X[:-1,:].float()
      self.target = X[1:,:-1].float()
      
  def __getitem__(self, index):
      x = self.data[index]
      y = self.target[index]
      return x, y
 
  def __len__(self):
      return len(self.data)
 
  def seq_data_gen(self, seq_size, input_dim, output_dim):
      Data = self.data.view(-1, seq_size, input_dim)
      Target = self.target.view(-1, seq_size, output_dim)
      data_seq = []
      target_seq = []
      for seq in range(seq_size):
        data_seq.append(Data[:,seq,:].tolist())
        target_seq.append(Target[:,seq,:].tolist())
      data_seq = torch.tensor(data_seq)
      target_seq = torch.tensor(target_seq)

      self.data = (data_seq[:,:,:]).float()
      self.target = (target_seq[:,:,:]).float() 
      return 

In [ ]:
# Load dataset
data=pd.read_csv('../../data/csv/Quadruped_data_100.txt')
data=data.drop("Num",axis=1)
Quad_data = torch.tensor(data.values)
Quad_data = Quad_data[:-1,:]
# Quad_data = torch.cat([Quad_data[:,:].view(-1,62), Quad_data[-1,:].view(-1,62)],dim=0)
print(np.shape(Quad_data))
data.head()

In [ ]:
# Data organizing
NUM_val=5000
input_dim = 61
output_dim = 60

tr_data = Quad_data[0:len(Quad_data)-NUM_val,:]
val_data= Quad_data[len(Quad_data)-NUM_val:,:]

# Generate input and target
tr_data = torch.cat([tr_data[:,:].view(-1,input_dim), tr_data[-1,:].view(-1,input_dim)],dim=0)
Tr_dataset = GenDataset(tr_data)
Xtr=Tr_dataset.data
Ytr=Tr_dataset.target

val_data = torch.cat([val_data[:,:].view(-1,input_dim), val_data[-1,:].view(-1,input_dim)],dim=0)
Val_dataset = GenDataset(val_data)
Xval=Val_dataset.data
Yval=Val_dataset.target


In [ ]:
seq_size=50 #약 1초

Tr_dataset.seq_data_gen(seq_size, input_dim, output_dim)
Xtr_LSTM=Tr_dataset.data
Ytr_LSTM=Tr_dataset.target

Val_dataset.seq_data_gen(seq_size, input_dim, output_dim)
Xval_LSTM=Val_dataset.data
Yval_LSTM=Val_dataset.target

print("Seqeuntial data generated!")


In [ ]:
# Upload data on GPU
# Xtr=Xtr.to(device)
# Xtr_LSTM=Xtr_LSTM.to(device)
# Xval=Xval.to(device)
# Xval_LSTM=Xval_LSTM.to(device)
print('Xtr      : ',np.shape(Xtr))
print('Xval     : ',np.shape(Xval))
print('Xtr_LSTM : ',np.shape(Xtr_LSTM))
print('Xval_LSTM: ',np.shape(Xval_LSTM))
print('\n')
# Ytr=Ytr.to(device)
# Ytr_LSTM=Ytr_LSTM.to(device)
# Yval=Yval.to(device)
# Yval_LSTM=Yval_LSTM.to(device)
print('Ytr      : ',np.shape(Ytr))
print('Yval     : ',np.shape(Yval))
print('Ytr_LSTM : ',np.shape(Ytr_LSTM))
print('Yval_LSTM: ',np.shape(Yval_LSTM))

<h1>Network definition</h1>

In [ ]:
class LSTMout(nn.Module):
    def forward(self, X):
        out_, _ = X
        return out_

class DeepQMP(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
    def __init__(self, input_dim, hidden_dim, output_dim): 
        super(DeepQMP, self).__init__() 

        # LSTM
        self.lstm1 = nn.LSTM(input_dim,     hidden_dim[0], bias=True)   
        self.lstm2 = nn.LSTM(hidden_dim[0], hidden_dim[1], bias=True) 
        dslayer1 = 64
        dslayer2 = 64
        self.myLSTM_mlp = nn.Sequential(
          nn.Linear(hidden_dim[1],  dslayer1, bias=True),   nn.ReLU(),
          nn.Linear(dslayer1,       dslayer2, bias=True),   nn.ReLU(),
          nn.Linear(dslayer2,       output_dim, bias=True), nn.ReLU()
        )

        # MLP
        nnlayer1 = 64
        nnlayer2 = 64
        self.myMLP = nn.Sequential(  
          nn.Linear(input_dim,  nnlayer1, bias=True),   nn.ReLU(),
          nn.Linear(nnlayer1,   nnlayer2, bias=True),   nn.ReLU(),
          nn.Linear(nnlayer2,   output_dim, bias=True), nn.ReLU()
        )

        # Weighted SUM
        self.End = nn.Linear(in_features=output_dim*2, out_features=output_dim, bias=False)  
      
    # def forward(self, X):
    def forward(self, X, HnC1, HnC2):
        # X : data, HnC : Hiddne and Cell state
        # LSTM
        lstm_out1, _ = self.lstm1(X, HnC1)
        lstm_out2, _ = self.lstm2(lstm_out1, HnC2)
        LSTMoutput = self.myLSTM_mlp(lstm_out2[-1])

        # MLP
        mlp_X = torch.cat([X[-1,:,0].view(-1,1), X[-1,:,1].view(-1,1)],dim=1)
        for inx in range(input_dim-2):
              mlp_X = torch.cat([mlp_X[:,0:inx+2], X[-1,:,inx+1].view(-1,1)],dim=1)

        NNoutput = self.myMLP(mlp_X)

        output_ = self.End(torch.cat([NNoutput.view(-1,output_dim), LSTMoutput.view(-1,output_dim)],dim=1))

        return output_
      
  

<h1>Training sequence</h1>

In [ ]:
# Network model
# model = Net();
hidden_dim=[64,64]
model = DeepQMP(input_dim, hidden_dim, output_dim); # (input dimension=7, hidden1 state dimension=1, hidden2 state dimension=1, output dimension=8)
# model = model.to(device)
# if device == 'cuda':
#     model = torch.nn.DataParallel(model)
#     cudnn.benchmark = True
# loss and optimizer
L1_loss = nn.L1Loss()
L2_loss = nn.MSELoss()
Huber_loss = nn.SmoothL1Loss()
# optimizer = torch.optim.SGD(params=model.parameters(), lr=0.00001, momentum=0.9, weight_decay =0.0001)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.002)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience= 10,mode='min')

In [ ]:
def L1_regularization(model):
    regularization_loss=0
    for param in model.parameters():
        regularization_loss += torch.sum(torch.abs(param))

    return regularization_loss
 
def L2_regularization(model):
    regularization_loss=0
    for param in model.parameters():
        regularization_loss += param.norm(2)**2

    return regularization_loss

def Loss(hypo, X, Y):
    # Clear display
    clear_output(wait=True)

    # alpha=0.01
    # lambdaExp = 1
    # lambdaRatio = 0.001
    # lambdaIN = 0.0005;
    # lambdaReg = 0.0001
    alpha=0.0001
    lambdaExp = 10
    lambdaRatio = 0.001
    lambdaIN = 1;
    lambdaReg = 0.1

    Lexp = L2_loss(input=hypo, target=Y)
    Lin = L2_loss(input=hypo, target=X[:,:-1])
    Lreg = L2_regularization(model)

    return lambdaExp*Lexp + lambdaRatio*Lexp/(alpha+lambdaIN*Lin) + lambdaReg*Lreg
    #   return Lexp
#     return lambdaExp*Lexp + lambdaIN/(alpha+Lin) + lambdaReg*Lreg

In [ ]:
# Learn
model.train()
epoch = 1000
batch=1000
batch_val=int(NUM_val/seq_size)
THRESH=0.001
 
hypothesis=[]
prediction=[]

train_loss=[]
validation_loss=[]
validation_step=[]

trloss=0
valloss=0

SRT=0;END=SRT+batch;

# Initialize H(X)
hidden1 = torch.zeros(1, batch, hidden_dim[0], requires_grad=True)#.to(device)
cell1 = torch.zeros(1, batch, hidden_dim[0], requires_grad=True)#.to(device)
hidden2 = torch.zeros(1, batch, hidden_dim[1], requires_grad=True)#.to(device)
cell2 = torch.zeros(1, batch, hidden_dim[1], requires_grad=True)#.to(device)

for step in tqdm(range(epoch), position=0, leave=True):
    # Sample among Training data
    Xo = Xtr[seq_size-1::seq_size,:]#.to(device)
    Yo = Ytr[seq_size-1::seq_size,:]#.to(device)
    hypothesis = model(Xtr_LSTM[:,SRT:END,:], (hidden1, cell1), (hidden2, cell2))
    trloss = Loss(hypothesis,Xo[SRT:END,:],Yo[SRT:END,:])
    # Optimize
    optimizer.zero_grad()
    trloss.backward()
    optimizer.step()
    scheduler.step(trloss)
    # Save train loss
    train_loss.append(trloss.data)

    # Validation
    if step%10 == 0:
        hidden_val1 = torch.zeros(1, batch_val, hidden_dim[0], requires_grad=False)#.to(device)
        cell_val1 = torch.zeros(1, batch_val, hidden_dim[0], requires_grad=False)#.to(device)
        hidden_val2 = torch.zeros(1, batch_val, hidden_dim[1], requires_grad=False)#.to(device)
        cell_val2 = torch.zeros(1, batch_val, hidden_dim[1], requires_grad=False)#.to(device)
        Xo = Xval[seq_size-1::seq_size,:]#.to(device)
        Yo = Yval[seq_size-1::seq_size,:]#.to(device)
        
        hypothesis_ = model(Xval_LSTM, (hidden_val1, cell_val1), (hidden_val2, cell_val2))
        valloss = Loss(hypothesis_,Xo[:,:],Yo)
        validation_step.append(step)
        validation_loss.append(valloss.data)
  
    # # Sequences    
    num_sam = 100
    plt.title("First "+str(num_sam)+" samples")
    plt.scatter(Xtr_LSTM[-1,SRT:SRT+num_sam,0].cpu(), Xtr_LSTM[-1,SRT:SRT+num_sam,1].cpu(),s=100.0, c='y', edgecolors='black', label='Start')
    plt.scatter(Ytr_LSTM[-1,SRT:SRT+num_sam,0].cpu(), Ytr_LSTM[-1,SRT:SRT+num_sam,1].cpu(),s=60.0 , c='r', edgecolors='black', label='Goal')
    plt.scatter(hypothesis[0:num_sam,0].cpu().detach().numpy(), hypothesis[0:num_sam,1].cpu().detach().numpy(),s=30.0 , c='c', alpha=0.8, label='Predict')
    plt.legend(fontsize='x-large')
    plt.savefig('./graph/Single_output.png',dpi=100)
    plt.show()

    # Plot and Save losses
    plt.title("Train loss : "+str(trloss.cpu().detach().numpy())+"\nValidation loss : "+str(valloss.cpu().detach().numpy()))
    loss1=plt.plot(train_loss, color='b', label='Train loss')
    loss2=plt.plot(validation_step, validation_loss , color='r', label='Validation loss')
    plt.xlabel("Steps")
    plt.ylabel("Losses")
    plt.legend()
    plt.savefig('./graph/LearningLoss.png',dpi=100)
    plt.show()
    # plt.clf()
    
     
    # Save checkpoint
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': trloss},
                "./model/DeepQMP.tar", pickle_protocol = 4)
                
    SRT=END
    if SRT+batch>=len(Xtr[seq_size-1::seq_size]):
        SRT=0
    
    END=SRT+batch;
    if END>=len(Xtr[seq_size-1::seq_size]):
        END=len(Xtr[seq_size-1::seq_size])
        
    if (trloss.data > 0.0) and (trloss.data < THRESH) and abs((valloss.detach().numpy() - trloss.detach().numpy()) < 1.0):
        break;
 
print("Final loss : ",trloss.data.item())
t=time.localtime(time.time())
plt.title("Train loss : "+str(trloss.cpu().detach().numpy()))
plt.plot(train_loss)
plt.savefig('./graph/LearningLoss_'+str(t.tm_year)+str(t.tm_mon)+str(t.tm_mday)+str(t.tm_hour)+str(t.tm_min)+'.png',dpi=100)
# plt.show()
 
# Save model & parameters
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': trloss},
            "./model/DeepQMP"+str(t.tm_year)+str(t.tm_mon)+str(t.tm_mday)+str(t.tm_hour)+str(t.tm_min)+".tar", pickle_protocol = 4)

In [ ]:
hidden1 = torch.zeros(1, 19900, hidden_dim[0], requires_grad=True)#.to(device)
cell1 = torch.zeros(1, 19900, hidden_dim[0], requires_grad=True)#.to(device)
hidden2 = torch.zeros(1, 19900, hidden_dim[1], requires_grad=True)#.to(device)
cell2 = torch.zeros(1, 19900, hidden_dim[1], requires_grad=True)#.to(device)

Xo = Xtr[seq_size-1::seq_size,:]#.to(device)
Yo = Ytr[seq_size-1::seq_size,:]#.to(device)
hypothesis = model(Xtr_LSTM, (hidden1, cell1), (hidden2, cell2))
XYdist = torch.norm(hypothesis_[:,0:2]-Yo[:,0:2], 1)
print(XYdist)

In [ ]:
a=[1,2,3,4,5,6,7,8,9]
s=3
print(a[s-1::s])

<h1>Total test</h1>

In [ ]:
# Eval
import copy
model = DeepQMP(input_dim, hidden_dim, output_dim);
# model = model.to(device)
# if device == 'cuda':
#     model = torch.nn.DataParallel(model)
#     cudnn.benchmark = True
params = torch.load("./model/DeepQMP.tar", map_location = "cpu")
model.load_state_dict(params['model_state_dict'])
model.eval()

err_x = []
err_y = []
err_yaw = []

hidden_test1 = torch.zeros(1, 1, hidden_dim[0], requires_grad=False)#.to(device)
cell_test1 = torch.zeros(1, 1, hidden_dim[0], requires_grad=False)#.to(device)
hidden_test2 = torch.zeros(1, 1, hidden_dim[1], requires_grad=False)#.to(device)
cell_test2 = torch.zeros(1, 1, hidden_dim[1], requires_grad=False)#.to(device)
 
index = np.random.randint(len(Xval[seq_size-1::seq_size,:]))
ts=int(round(time.time() * 1000))
input = Xval_LSTM[:,index,:]

Xo=Xval[seq_size-1::seq_size,:]
hypo = model(input.view(seq_size,-1,input_dim),(hidden_test1, cell_test1),(hidden_test2, cell_test2))
output=copy.deepcopy(Variable(hypo))
output[:,0:2] = hypo[:,0:2]
 
tf=int(round(time.time() * 1000))
print(tf-ts,'ms')
 
plt.title("Arbitrary sample - XY")
# plt.xlim(-22,22);plt.ylim(-22,22)
plt.scatter(Xval_LSTM[:,index,0].cpu(), Xval_LSTM[:,index,1].cpu(),s=500.0, c='y', edgecolors='y', label='XY input')
plt.scatter(Yval_LSTM[:,index,0].cpu(), Yval_LSTM[:,index,1].cpu(),s=200.0, c='r', edgecolors='r', label='XY expected')
plt.scatter(output[:,0].item(), output[:,1].item(),s=100.0, c='b', edgecolors='black', linewidth=5, alpha=0.8 , label='XY predicted')
plt.scatter(Xval_LSTM[0,index,0].cpu(), Xval_LSTM[0,index,1].cpu(),s=700.0, c='m', edgecolors='y', label='XY origin')
plt.scatter(Xval_LSTM[-1,index,0].cpu(), Xval_LSTM[-1,index,1].cpu(),s=700.0, c='g', edgecolors='r', label='XY last')
plt.legend(fontsize='x-large')
plt.show()

In [ ]:
input = Xval[index,:]
 
hidden_test1 = torch.zeros(1, 1, hidden_dim1).to(device)
cell_test1 = torch.zeros(1, 1, hidden_dim1).to(device)
hidden_test2 = torch.zeros(1, 1, hidden_dim2).to(device)
cell_test2 = torch.zeros(1, 1, hidden_dim2).to(device)

Xo=Xval
hypo,(h1,c1),(h2,c2) = model(input.view(1,-1,input_dim),(hidden_test1, cell_test1),(hidden_test2, cell_test2))
output=copy.deepcopy(Variable(hypo))
output[:,0:2] = hypo[:,0:2]
 
plt.title("Arbitrary sample - XY")
# plt.xlim(-22,22);plt.ylim(-22,22)
plt.scatter(Xval[index,0].cpu(), Xval[index,1].cpu(),s=500.0, c='y', edgecolors='y', label='XY origin')
plt.scatter(Yval[index,0].cpu(), Yval[index,1].cpu(),s=300.0, c='r', edgecolors='y', label='XY expected')
plt.scatter(output[:,0].item(), output[:,1].item(),s=100.0, c='b', edgecolors='black', linewidth=5, alpha=0.8 , label='XY predicted')
plt.legend(fontsize='x-large')
plt.show()

In [ ]:
batch_test = 1000
# input = Xte.reshape(input_dim,-1,1)
# input = Xte.reshape(3,-1,1)
input = Xte_LSTM
# input = Xte_ori
hidden_test1 = torch.zeros(1, NUM_val, hidden_dim1)#.to(device)
cell_test1 = torch.zeros(1, NUM_val, hidden_dim1)#.to(device)
hidden_test2 = torch.zeros(1, NUM_val, hidden_dim2)#.to(device)
cell_test2 = torch.zeros(1, NUM_val, hidden_dim2)#.to(device)
hypo,(h1,c1),(h2,c2)  = model(input,(hidden_test1, cell_test1),(hidden_test2, cell_test2))
output=copy.deepcopy(Variable(hypo))
output[:,0:2] = hypo[:,0:2]/scaler + Xte_ori[:,0:2].to(device)
output[:,2] = hypo[:,2] + Xte_ori[:,2].to(device)

error_input = np.sqrt((np.square(Xte_ori[:,0:-2].cpu().numpy() - output.detach().cpu().numpy())).mean())
error = (output-Yte_ori).cpu().detach().numpy()
for i in range(len(error)):
        if abs(error[i,2]) > 180:
          error[i,2] = np.sign(error[i,2])*(360-abs(error[i,2]));
error_target = np.sqrt(np.square(error).mean())

Pred = output.cpu().detach().numpy()
 
num_sam = 100
plt.title(str(num_sam)+ " samples. vs. Input - XY")
plt.scatter(Xte_LSTM_ori[0,0:num_sam,0].cpu(), Xte_LSTM_ori[1,0:num_sam,0].cpu(),s=50.0, c='y', edgecolors='black')
plt.scatter(Pred[0:num_sam,0], Pred[0:num_sam,1],s=50.0 , c='c', edgecolors='black', alpha=0.8)
plt.show()
plt.title(str(num_sam)+ " samples. vs. Target - XY")
plt.scatter(Yte_LSTM_ori[0,0:num_sam,0].cpu(), Yte_LSTM_ori[1,0:num_sam,0].cpu(),s=50.0 , c='r', edgecolors='black')
plt.scatter(Pred[0:num_sam,0], Pred[0:num_sam,1],s=50.0 , c='c', edgecolors='black', alpha=0.8)
plt.show()

plt.title(str(num_sam)+ " samples. vs. Input - Yaw")
plt.plot(Xte_LSTM_ori[2,0:num_sam,0].cpu(),'.', c='y')
plt.plot(Pred[0:num_sam,2],'.', c='c', alpha=0.8)
plt.show()
plt.title(str(num_sam)+ " samples. vs. Target - Yaw")
plt.plot(Yte_LSTM_ori[2,0:num_sam,0].cpu(),'.', c='r')
plt.plot(Pred[0:num_sam,2],'.', c='c', alpha=0.8)
plt.show()

plt.title(str(num_sam)+ " samples. vs. Input - dx")
Xte_ori=Xte_ori.cpu()
plt.ylim(min(np.min(Pred[0:num_sam,0]-Xte_ori[0:num_sam,0].numpy()),np.min(error[0:num_sam,0])),max(np.max(Pred[0:num_sam,0]-Xte_ori[0:num_sam,0].numpy()),np.max(error[0:num_sam,0])))
plt.plot(Pred[0:num_sam,0]-Xte_ori[0:num_sam,0].numpy(),'.', c='y')
plt.title(str(num_sam)+ " samples. vs. Target - dx")
plt.ylim(min(np.min(Pred[0:num_sam,0]-Xte_ori[0:num_sam,0].numpy()),np.min(error[0:num_sam,0])),max(np.max(Pred[0:num_sam,0]-Xte_ori[0:num_sam,0].numpy()),np.max(error[0:num_sam,0])))
plt.plot(error[0:num_sam,0],'.', c='r')
plt.show()

plt.title(str(num_sam)+ " samples. vs. Input - dy")
plt.ylim(min(np.min(Pred[0:num_sam,1]-Xte_ori[0:num_sam,1].numpy()),np.min(error[0:num_sam,1])),max(np.max(Pred[0:num_sam,1]-Xte_ori[0:num_sam,1].numpy()),np.max(error[0:num_sam,1])))
plt.plot(Pred[0:num_sam,1]-Xte_ori[0:num_sam,1].numpy(),'.', c='y')
plt.title(str(num_sam)+ " samples. vs. Target - dy")
plt.ylim(min(np.min(Pred[0:num_sam,1]-Xte_ori[0:num_sam,1].numpy()),np.min(error[0:num_sam,1])),max(np.max(Pred[0:num_sam,1]-Xte_ori[0:num_sam,1].numpy()),np.max(error[0:num_sam,1])))
plt.plot(error[0:num_sam,1],'.', c='r')
plt.show()

plt.title(str(num_sam)+ " samples. vs. Input - dYaw")
plt.ylim(min(np.min(Pred[0:num_sam,2]-Xte_ori[0:num_sam,2].numpy()),np.min(error[0:num_sam,2])),max(np.max(Pred[0:num_sam,2]-Xte_ori[0:num_sam,2].numpy()),np.max(error[0:num_sam,2])))
plt.plot(Pred[0:num_sam,2]-Xte_ori[0:num_sam,2].numpy(),'.', c='y')
plt.title(str(num_sam)+ " samples. vs. Target - dYaw")
plt.ylim(min(np.min(Pred[0:num_sam,2]-Xte_ori[0:num_sam,2].numpy()),np.min(error[0:num_sam,2])),max(np.max(Pred[0:num_sam,2]-Xte_ori[0:num_sam,2].numpy()),np.max(error[0:num_sam,2])))
plt.plot(error[0:num_sam,2],'.', c='r')
plt.show()

# plt.title(str(num_sam)+ " sample - X")
# plt.plot(Yte_LSTM_ori[0,0:num_sam,0],'.', c='r')
# plt.plot(Pred[0:num_sam,0],'.', c='c', alpha=0.8)
# plt.show()
# plt.title(str(num_sam)+ " sample - Y")
# plt.plot(Yte_LSTM_ori[1,0:num_sam,0],'.', c='r')
# plt.plot(Pred[0:num_sam,1],'.', c='c', alpha=0.8)
# plt.show()
 
Yte_LSTM_ori=Yte_LSTM_ori.cpu()
plt.title("Expected - Predicted - X")
plt.scatter(Yte_LSTM_ori[0,:,0], Pred[:,0], s=20.0 , c='r', edgecolors='black')
plt.show()
 
plt.title("Expected - Predicted - Y")
plt.scatter(Yte_LSTM_ori[1,:,0], Pred[:,1], s=20.0 , c='g', edgecolors='black')
plt.show()
 
plt.title("Expected - Predicted - YAW")
plt.scatter(Yte_LSTM_ori[2,:,0], Pred[:,2], s=20.0 , c='c', edgecolors='black')
plt.show()
 
print("TEST error     : ",error_target)
print("TEST error_in  : ",error_input)
print("TEST dist      : ", np.sqrt((np.square(Yte_ori[:,0:2].cpu().numpy() - output[:,0:2].cpu().detach().numpy())).mean()))
print("TEST dist_in   : ", np.sqrt((np.square(Xte_ori[:,0:2].cpu().numpy() - output[:,0:2].cpu().detach().numpy())).mean()))

print("TEST dyaw      : ", np.sqrt(np.square(error[:,2].mean())))
print("TEST dyaw_in   : ", np.sqrt((np.square(Xte_ori[:,2].cpu().numpy() - output[:,2].cpu().detach().numpy())).mean()))

print("TEST dvel      : ", np.sqrt((np.square(Yte_ori[:,3:6].cpu().numpy() - output[:,3:6].cpu().detach().numpy())).mean()))
print("TEST delv_in   : ", np.sqrt((np.square(Xte_ori[:,3:6].cpu().numpy() - output[:,3:6].cpu().detach().numpy())).mean()))

print("TEST domg      : ", np.sqrt((np.square(Yte_ori[:,6:8].cpu().numpy() - output[:,6:8].cpu().detach().numpy())).mean()))
print("TEST domg_in   : ", np.sqrt((np.square(Xte_ori[:,6:8].cpu().numpy() - output[:,6:8].cpu().detach().numpy())).mean()))

print('TEST input     : ',Xte_LSTM_ori[0:3,index,:].T.cpu().detach().numpy()[0])
print('TEST expected  : ',Yte_LSTM_ori[0:3,index,:].T.cpu().detach().numpy()[0])
print('TEST predict   : ',Pred[index,0:3])

In [ ]:
print("Training data")
plt.title("Training - XY")
plt.scatter(Xtr_LSTM_ori[0,:,:], Xtr_LSTM_ori[1,:,:],s=20.0 , c='r', edgecolors='black')
plt.show()
plt.title("Training - X")
plt.plot(Xtr_LSTM_ori[0,:,:],'.', c='c')
plt.show()
plt.title("Training - Y")
plt.plot(Xtr_LSTM_ori[1,:,:],'.', c='c')
plt.show()
plt.title("Training - YAW")
plt.plot(Xtr_LSTM_ori[2,:,:],'.', c='c')
plt.show()
plt.title("Training - V")
plt.scatter(Xtr_LSTM_ori[3,:,:], Xtr_LSTM_ori[4,:,:],s=20.0 , c='y', edgecolors='black')
plt.show()
plt.title("Training - Vx")
plt.plot(Xtr_ori[:,3],'.', c='g')
plt.show()
plt.title("Training - Vy")
plt.plot(Xtr_ori[:,4],'.', c='g')
plt.show()
plt.title("Training - Vyaw")
plt.plot(Xtr_ori[:,5],'.', c='g')
plt.show()
plt.title("Training - wR")
plt.plot(Xtr_ori[:,6],'.', c='g')
plt.show()
plt.title("Training - wL")
plt.plot(Xtr_ori[:,7],'.', c='g')
plt.show()
plt.title("Training - tR")
plt.plot(Xtr_ori[:,8],'.', c='g')
plt.show()
plt.title("Training - tL")
plt.plot(Xtr_ori[:,9],'.', c='g')
plt.show()

In [1]:
import torch 
import pandas as pd
import numpy as np
from tqdm import tqdm

data = pd.read_csv('../../data/csv/Hound_data.txt')

Quad_data = torch.tensor(data.values)
# Quad_data = Quad_data[:-1,:]
print(np.shape(Quad_data))
data

torch.Size([1987767, 42])


,PathNum,NodeNum,baseX,baseY,baseZ,baseR,baseP,baseYaw,base_dX,base_dY,...,F_LHx,F_LHy,F_LHz,F_RHx,F_RHy,F_RHz,LF,RF,LH,RH
0,0,1,0.000000,0.000000,0.500000,0.000000,-0.000000,0.000000,0.000000,0.000000,...,1.711860,0.607218,42.2079,-0.531854,0.006337,23.14820,1,1,1,1
1,0,2,0.000007,-0.000015,0.499853,0.001681,0.000133,-0.001566,0.001424,-0.003399,...,1.533830,0.544067,42.6198,-0.427283,-0.181002,23.35030,1,1,1,1
2,0,3,0.000031,-0.000074,0.499433,0.006295,0.000469,-0.005890,0.003489,-0.008634,...,1.109290,0.393477,43.6022,-0.146997,-0.680728,23.93650,1,1,1,1
3,0,4,0.000079,-0.000194,0.498777,0.013196,0.000916,-0.012410,0.006194,-0.015706,...,0.602576,0.213741,44.7748,0.272096,-1.427310,24.82910,1,1,1,1
4,0,5,0.000157,-0.000394,0.497918,0.021740,0.001379,-0.020566,0.009539,-0.024614,...,0.178034,0.063151,45.7571,0.793089,-2.355210,25.95050,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987762,15976,77,0.622705,1.090240,0.522588,0.011927,0.019628,1.334300,0.257720,1.348550,...,0.000000,0.000000,0.0000,-3.090640,-10.699900,72.49340,1,0,0,1
1987763,15976,78,0.630351,1.131170,0.528825,0.019917,0.020489,1.361290,0.252653,1.381260,...,0.000000,0.000000,0.0000,4.325180,23.326400,39.87370,1,0,0,1
1987764,15976,79,0.635381,1.159000,0.531467,0.026251,0.017863,1.382130,0.250533,1.401050,...,0.000000,0.000000,0.0000,2.005350,10.391900,12.56110,1,0,0,1
1987765,15976,80,0.637882,1.173050,0.532108,0.029318,0.015757,1.393550,0.249672,1.409090,...,0.000000,0.000000,0.0000,0.616845,3.168020,3.46435,1,0,0,1


In [2]:
path=[0]*data.PathNum.max()
path_idx=-1

for idx in tqdm(range(len(Quad_data)), position=0, leave=True):
  if path_idx+1 == Quad_data[idx,0]:
    # Increase path index
    tmp=[]
    path_idx = path_idx+1
    if path_idx == data.PathNum.max():
      # Skip last path
      break;
  if Quad_data[idx,0]==path_idx:
    # Rearrange data into (path, samples, variables)
    tmp.append(Quad_data[idx,2:].tolist())
    path[path_idx]=tmp


100%|█████████▉| 1987686/1987767 [02:03<00:00, 16071.52it/s]
